In [102]:
import matplotlib.pyplot as plt
import plotly.express as px
import utils
import importlib
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.ticker as plticker
importlib.reload(utils)
import datetime as dt
import numpy as np
import warnings
import os
import pandas as pd
from tqdm import tqdm
from pandas.errors import SettingWithCopyWarning
# from pandas.core.common import SettingWithCopyWarning
import dateparser

# Ignore dateparser warnings regarding pytz
warnings.filterwarnings(
    "ignore",
    message="The localize method is no longer necessary, as this time zone supports the fold attribute",
)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [103]:
import glob

breakdowns_arr = []
breakdown_time = '10:00:00'
listfiles = glob.glob('MCTS_RESULTS/2022_NEW_C/*.log')
for lf in tqdm(listfiles):
    fname = os.path.basename(lf)
    date = fname.split("_")[2]
    df = utils.get_bus_df(lf)
    df_arr = []
    for k, v in df.groupby('trip_id'):
        v['sequence'] = list(range(len(v)))
        v['last_sequence'] = len(v)
        df_arr.append(v)
    df = pd.concat(df_arr)
    df['count'] = 1
    _df = df.groupby('bus_id').agg({'count':'sum', 'got_on_bus':'sum'})
    _df = _df[~_df.index.isin(['41', '42', '43', '44', '45'])].sort_values('got_on_bus', ascending=False)
    # display(df)
    most_used_bus = _df.iloc[0].name
    a = df.query("bus_id == @most_used_bus")
    dt = f"{date} {breakdown_time}"
    dt = dateparser.parse(dt, date_formats=["%Y%m%d %H:%M:%S"])
    # print(dt)
    
    minidx = (abs(dt - a['scheduled_time'])).idxmin()
    b = a.loc[[minidx]].iloc[0]
    if b.sequence == b.last_sequence:
        minidx = minidx + 5
    break_sched = a.loc[[minidx]].scheduled_time.iloc[0]
    breakdowns_arr.append((date, most_used_bus, break_sched))
breakdowns_df = pd.DataFrame(breakdowns_arr, columns=['date', 'bus', 'datetime'])

100%|██████████| 31/31 [03:31<00:00,  6.83s/it]


In [105]:
breakdowns_df.to_pickle('results/breakdowns_2022_10.pkl')